# **Import Libraries and Set Seed**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import time
import glob
import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
import cv2
import shutil

In [ ]:
from numpy.random import seed
seed(42)
from tensorflow.random import set_seed
set_seed(42)

random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)

# **Data Preprocessing**

In [ ]:
df = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')

In [ ]:
df.head()

In [ ]:
X_train = df.copy().drop(['target', 'id'], axis=1)

Y_train = df.target.copy()

In [ ]:
class_map = {'Class_1': 0,
            'Class_2': 1,
            'Class_3': 2,
            'Class_4': 3,
            'Class_5': 4,
            'Class_6': 5,
            'Class_7': 6,
            'Class_8': 7,
            'Class_9': 8}
Y_train = Y_train.map(class_map).astype('int')
Y_train = tf.keras.utils.to_categorical(Y_train)

In [ ]:
for column in X_train.columns:
    X_train[column] = (X_train[column] -
                           X_train[column].mean()) / X_train[column].std()  

In [ ]:
X_train

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, random_state=0, train_size=0.95)

In [ ]:
X_train = tf.expand_dims(X_train, -1)
X_valid = tf.expand_dims(X_valid, -1)

# **Model Architecture**

In [ ]:

model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=75, kernel_size=3, strides=(2), activation='relu', input_shape=[75,1]),
    tf.keras.layers.Conv1D(filters=24, kernel_size=2, strides=(2), activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(9, activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# **Callbacks**

In [ ]:
patience = 2
stop_patience = 5
factor = 0.5

callbacks = [
    tf.keras.callbacks.ModelCheckpoint("classify_model.h5", save_best_only=True, verbose = 0),
    tf.keras.callbacks.EarlyStopping(patience=stop_patience, monitor='val_loss', verbose=1, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=factor, patience=patience, verbose=1)
]

# **Model Training**

In [ ]:
epochs=100
batch_size=256

In [ ]:
history = model.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), epochs=epochs, batch_size=batch_size, callbacks=callbacks)

In [ ]:
test_df = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')

In [ ]:
for column in test_df.columns[1:]:
    test_df[column] = (test_df[column] -
                           test_df[column].mean()) / test_df[column].std()  

In [ ]:
test_df

In [ ]:
X_test = test_df.copy().drop(['id'], axis=1)

In [ ]:
X_test = tf.expand_dims(X_test, -1)

In [ ]:
predictions = model.predict(X_test)
predictions = pd.DataFrame(predictions, columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9'])
predictions

In [ ]:
id_column = []
for i in range(100000):
    id_column.append(i + 200000)

In [ ]:
id_column = pd.Series(id_column)

In [ ]:
predictions = pd.concat([id_column, predictions], axis=1)

In [ ]:
predictions = predictions.rename(columns={0:'id'})

In [ ]:
predictions

In [ ]:
predictions.to_csv('predictions.csv', index=False)